#### 📌 **HİPERPARAMETRELERİ BURADAN AYARLA**

In [1]:
learning_rate = 1e-4
dropout_rate = 0.6
batch_size = 64
filters = [32, 64, 128, 256]
earlystop_patience = 14
reduce_lr_patience = 9
epochs = 40
target_size = (224, 224)

In [2]:
import os
import re
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (classification_report, confusion_matrix, f1_score,
                             accuracy_score, roc_auc_score, precision_score, recall_score)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, ReLU,
                                     MaxPooling2D, Dropout, Flatten, Dense)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

#### 📁 **1. Görüntü Yolu Listeleme Fonksiyonu**


In [ ]:
#chatgpt
def mammo_gorunumlerini_listele(kok_dizin):
    class_dirs = {
        'normal': ['Birad1'],
        'benign': ['Birad3'],
        'cancer': ['Birad4', 'Birad5'],
    }
    paths, labels = [], []
    for cls, subdirs in class_dirs.items():
        for sd in subdirs:
            dir_path = os.path.join(kok_dizin, sd)
            if not os.path.isdir(dir_path):
                continue
            for fname in sorted(os.listdir(dir_path)):
                if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                    paths.append(os.path.join(dir_path, fname))
                    labels.append(cls)
    return paths, labels

#### 🧼 **2. Görüntü Ön İşleme**


In [ ]:
#chatgpt ile ayırdım
def read_grayscale_image(path, target_size=target_size):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, target_size)
    return img.astype(np.float32)

def normalize_image(img, method='minmax'):
    if method == 'minmax':
        return img / 255.0
    else:
        return (img - img.mean()) / (img.std() + 1e-8)

def convert_to_dataset(paths, labels, target_size=target_size, method='minmax'):
    X, y = [], []
    label_map = {'normal': 0, 'benign': 1, 'cancer': 2}
    for p, lbl in zip(paths, labels):
        img = read_grayscale_image(p, target_size)
        img = normalize_image(img, method)
        X.append(img[..., np.newaxis])
        y.append(label_map[lbl])
    return np.array(X), np.array(y)

#### 🧠 **3. CNN Model Tanımı**

In [5]:
def _conv_block(inputs, filters):
    x = Conv2D(filters, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
def create_cnn_model(input_shape=(224, 224, 1), num_classes=3, filters=filters, dropout_rate=dropout_rate):
    inp = Input(shape=input_shape)
    x = Conv2D(filters[0], (6, 6), padding='same')(inp)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)

    x = _conv_block(x, filters[1])
    x = _conv_block(x, filters[1])
    x = MaxPooling2D((2, 2))(x)
    
    x = _conv_block(x, filters[2])
    x = _conv_block(x, filters[2])
    x = MaxPooling2D((2, 2))(x)
    
    x = _conv_block(x, filters[3])
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    out = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=inp, outputs=out)

#### 🚂 **4. Model Eğitimi ve Kaydetme**


In [ ]:
if __name__ == '__main__':
    kok_dizin = os.path.join(os.getcwd(), "Project2")
    yollar, etiketler = mammo_gorunumlerini_listele(kok_dizin)
    X, y = convert_to_dataset(yollar, etiketler,target_size=target_size)
    X_egitim_val, X_test, y_egitim_val, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
    np.save("X_test_2_3.npy", X_test)
    np.save("y_test_2_3.npy", y_test)
    model = create_cnn_model()
    model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    geri_cagirmalar = [
        EarlyStopping(patience=earlystop_patience, restore_best_weights=True),
        ReduceLROnPlateau(patience=reduce_lr_patience, factor=0.5)
    ]
    #chatgpt
    model.fit(X_egitim_val, pd.get_dummies(y_egitim_val),
          epochs=epochs, batch_size=batch_size,validation_split=0.1,
          callbacks=geri_cagirmalar, verbose=1)

    olasiliklar = model.predict(X_test)
    esikler = np.linspace(0.4, 0.6, 21)
    en_iyi_f1, en_iyi_T_m, en_iyi_T_b = 0, 0.51, 0.49
    for T_m_aday in esikler:
        for T_b_aday in esikler:
            tahminler = [2 if p[2] >= T_m_aday else 1 if p[1] >= T_b_aday else 0 for p in olasiliklar]
            f1 = f1_score(y_test, tahminler, average='macro')
            if f1 > en_iyi_f1:
                en_iyi_f1, en_iyi_T_m, en_iyi_T_b = f1, T_m_aday, T_b_aday

Epoch 1/40
31/31 [==============================] - 11s 214ms/step - loss: 0.9102 - accuracy: 0.8110 - val_loss: 0.9711 - val_accuracy: 0.7944
Epoch 2/40
31/31 [==============================] - 5s 174ms/step - loss: 0.3549 - accuracy: 0.8764 - val_loss: 0.8976 - val_accuracy: 0.7944
Epoch 3/40
31/31 [==============================] - 5s 174ms/step - loss: 0.2717 - accuracy: 0.8936 - val_loss: 0.7137 - val_accuracy: 0.7944
Epoch 4/40
31/31 [==============================] - 5s 174ms/step - loss: 0.2720 - accuracy: 0.8904 - val_loss: 0.7377 - val_accuracy: 0.7944
Epoch 5/40
31/31 [==============================] - 5s 175ms/step - loss: 0.2405 - accuracy: 0.9019 - val_loss: 0.7747 - val_accuracy: 0.7944
Epoch 6/40
31/31 [==============================] - 5s 175ms/step - loss: 0.2318 - accuracy: 0.9050 - val_loss: 0.6746 - val_accuracy: 0.7944
Epoch 7/40
31/31 [==============================] - 5s 175ms/step - loss: 0.2281 - accuracy: 0.9050 - val_loss: 0.7829 - val_accuracy: 0.7944
Epoch

 Metirkler**

In [7]:
nihai_tahminler = [2 if p[2] >= en_iyi_T_m else 1 if p[1] >= en_iyi_T_b else 0 for p in olasiliklar]
print(f"En iyi eşik sınırı: T_m={en_iyi_T_m:.2f}, T_b={en_iyi_T_b:.2f}")
print(classification_report(y_test, nihai_tahminler, target_names=['normal','benign','cancer']))
print(confusion_matrix(y_test, nihai_tahminler))
print("Accuracy:", accuracy_score(y_test, nihai_tahminler))
print("Precision:", precision_score(y_test, nihai_tahminler, average='macro'))
print("Recall:", recall_score(y_test, nihai_tahminler, average='macro'))
try:
    print("AUC:", roc_auc_score(pd.get_dummies(y_test), olasiliklar, average='macro', multi_class='ovr'))
except:
    print("AUC hesaplanamadı (muhtemelen tek sınıf tahmini nedeniyle)")
with open("optimized_thresholds.txt", "w") as f:
    f.write(f"Best T_m: {en_iyi_T_m}\nBest T_b: {en_iyi_T_b}\nBest F1: {en_iyi_f1}")

En iyi eşik sınırı: T_m=0.40, T_b=0.40
              precision    recall  f1-score   support

      normal       0.99      0.97      0.98       187
      benign       0.71      0.95      0.81        39
      cancer       1.00      0.25      0.40        12

    accuracy                           0.93       238
   macro avg       0.90      0.72      0.73       238
weighted avg       0.94      0.93      0.92       238

[[181   6   0]
 [  2  37   0]
 [  0   9   3]]
Accuracy: 0.9285714285714286
Precision: 0.9002031665966093
Recall: 0.7222107957402075
AUC: 0.9566798038207818


In [8]:
#### 💾 **6. Model ve Eşik Kayıtları**


In [9]:
model_name = "model3_2.h5"
model.save(model_name)
print(f"Model kaydedildi: {model_name}")

Model kaydedildi: model3_2.h5
